In [1]:
import pandas as pd
import numpy as np
import os
import requests
import sqlite3

from datetime import date

from time import sleep 
from random import randint

from typing import Dict, List 
import re 


# Add parameters

In [2]:
today=str(date.today())
exp_file_name=("CB_landplots_sale_"+today+".gzip")
exp_file_path=str(os.path.dirname(os.path.realpath("__file__")))
print(exp_file_path+"/"+exp_file_name)

C:\Users\novot\OneDrive\Plocha\Project\Vojta/CB_landplots_sale_2023-01-23.gzip


In [3]:
category_main_cb = 3 # 1=byty, 2=domy, 3=pozemky, 4=komerční, 5=ostatní
category_type_cb = 1 # 1=prodej, 2=nájem, 3=dražba
category_sub_cb = [] # 34=garáže, 52=garážové stání
locality_region_id = [10] #10=Praha, 11=Středočeský kraj, 5: Liberecký kraj, 1: Českobudějovický kraj

# Code 

### Číselníky:

In [4]:
#https://1.im.cz/seznam/blog/articles/import.pdf
category_main_cb_dict={1:"byty", 2:"domy", 3:"pozemky", 4:"komerční", 5:"ostatní"}
category_main_cb_dict=pd.DataFrame.from_dict(category_main_cb_dict, orient="index", columns=["category_main_cb"])
print(category_main_cb_dict)

category_sub_cb_dict={
    2:"1+kk", 
    3:"1+1", 
    4:"2+kk", 
    5:"2+2", 
    6:"2+kk",
    34:"garáž",
    52:"garážové stání"}
category_sub_cb_dict=pd.DataFrame.from_dict(category_sub_cb_dict, orient="index", columns=["category_sub_cb"])
print(category_sub_cb_dict)

category_type_cb_dict={1:"prodej", 2:"nájem", 3:"dražba"}
category_type_cb_dict=pd.DataFrame.from_dict(category_type_cb_dict, orient="index", columns=["category_type_cb"])
print(category_type_cb_dict)

locality_region_id_dict={10:"Praha", 11:"Středočeský kraj", 5:"Liberecký kraj"}
locality_region_id_dict=pd.DataFrame.from_dict(locality_region_id_dict, orient="index", columns=["locality_region_id"])
print(locality_region_id_dict)

  category_main_cb
1             byty
2             domy
3          pozemky
4         komerční
5          ostatní
   category_sub_cb
2             1+kk
3              1+1
4             2+kk
5              2+2
6             2+kk
34           garáž
52  garážové stání
  category_type_cb
1           prodej
2            nájem
3           dražba
   locality_region_id
10              Praha
11   Středočeský kraj
5      Liberecký kraj


In [5]:
def download_lists(category_main_cb, category_type_cb, category_sub_cb, locality_region_id):

    category_sub_cb_string = '%7C'.join(str(v) for v in category_sub_cb)
    locality_region_id_string = '%7C'.join(str(v) for v in locality_region_id)

    collector={}
    i=0
    run=True

    while run==True:
        
        if len(category_sub_cb)>0:
            base_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb={}&category_sub_cb={}category_type_cb={}&locality_region_id={}&per_page60&page={}'.format(category_main_cb, category_sub_cb_string, category_type_cb, locality_region_id_string, i)
        else:
            base_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb={}&category_type_cb={}&locality_region_id={}&per_page60&page={}'.format(category_main_cb, category_type_cb, locality_region_id_string, i)
        
        r = requests.get(base_url)
        print("starting sleep")
        sleep(randint(1,3))

        if r.status_code==404:
            break
        elif r.status_code==200:
            r_dict=r.json()

            if len(r_dict["_embedded"]["estates"]) == 0:
                print(f"Page {i+1} is blank.")
                break

            collector[i]=r_dict
            
            print(f"Page {i+1} was scraped.")

            i=i+1
            j=0

        else:
            if  j==3:
                print(f"Code {r.status_code} was returned.")
                break
            else:
                j=j+1
    return collector

collector=download_lists(category_main_cb=category_main_cb, 
category_type_cb=category_type_cb, 
category_sub_cb=category_sub_cb, 
locality_region_id=locality_region_id)

starting sleep
Page 1 was scraped.
starting sleep
Page 2 was scraped.
starting sleep
Page 3 was scraped.
starting sleep
Page 4 was scraped.
starting sleep
Page 5 was scraped.
starting sleep
Page 6 was scraped.
starting sleep
Page 7 was scraped.
starting sleep
Page 8 was scraped.
starting sleep
Page 9 was scraped.
starting sleep
Page 10 was scraped.
starting sleep
Page 11 was scraped.
starting sleep
Page 12 was scraped.
starting sleep
Page 13 was scraped.
starting sleep
Page 14 was scraped.
starting sleep
Page 15 was scraped.
starting sleep
Page 16 was scraped.
starting sleep
Page 17 was scraped.
starting sleep
Page 18 was scraped.
starting sleep
Page 19 was scraped.
starting sleep
Page 20 was scraped.
starting sleep
Page 21 was scraped.
starting sleep
Page 22 was scraped.
starting sleep
Page 23 is blank.


In [6]:
def get_gps_lat_lon(estate_raw: Dict):
    gps_ = estate_raw['gps']
    return gps_['lat'], gps_['lon']

# more robust implementation using regex - https://docs.python.org/3/library/re.html
def get_area_from_name(name: str):
    """(\d*) matches any decimal digits repeated after each other"""
    name_ = name.split()
    return int(''.join(re.findall('(\d*)', ''.join(name_))))

In [7]:
collector[0];

In [8]:
# slow helper functions (use itertools.chains for speedup)
def flatten_list(l:List):
    return [item for sublist in l for item in sublist]

def get_distinct_nearby_objects(estates_individual_raw: Dict):
    lists_of_nearby_objects = [e['labelsAll'][1] for e in estates_individual_raw.values()]
    return list(set(flatten_list(lists_of_nearby_objects)))

def get_distinct_estate_attributes(estates_individual_raw: Dict):
    lists_of_estates_attributes = [e['labelsAll'][0] for e in estates_individual_raw.values()]
    return list(set(flatten_list(lists_of_estates_attributes)))

In [9]:
def get_company_details(estate_raw: Dict):
    try:
            company_id = estate_raw["_embedded"]["company"]["id"]
            company_name = estate_raw["_embedded"]["company"]["name"]
    except (KeyError):
            company_id = np.nan
            company_name = np.nan

    return company_id, company_name

In [10]:
def decode_collector(collector):
    estates_individual = {}

    for page, r in collector.items():
        for estate in r['_embedded']['estates']: 


            estate_relevant = pd.Series(dtype="object")

            estate_relevant['price'] = int(estate['price'])
            estate_relevant['price_czk'] = int(estate['price_czk']["value_raw"])
            estate_relevant['price_czk_unit'] = estate['price_czk']["unit"]
            estate_relevant['price_czk_name'] = estate['price_czk']["name"]
            estate_relevant['area'] = get_area_from_name(estate['name'])

            lat, lon = get_gps_lat_lon(estate)
            estate_relevant.loc['lat'] = lat
            estate_relevant.loc['lon'] = lon
            estate_relevant['locality'] = estate['locality']
            estate_relevant['type'] = estate['type']
            estate_relevant['category'] = estate['category']
            estate_relevant['is_auction'] = estate['is_auction']
            estate_relevant['exclusively_at_rk'] = estate['exclusively_at_rk']

            estate_relevant['category_main_cb'] = estate["seo"]["category_main_cb"]
            estate_relevant['category_sub_cb'] = estate["seo"]["category_sub_cb"]
            estate_relevant['category_type_cb'] = estate["seo"]["category_type_cb"]

            company_id, company_name = get_company_details(estate)
            estate_relevant['company_id'] = company_id
            estate_relevant['company_name'] = company_name
            
            estates_individual[estate['hash_id']] = estate_relevant
    
    # transforming into pandas df
    df = pd.concat(estates_individual).unstack()
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'hash_id'})
    
    return df

df=decode_collector(collector)

In [11]:
df.head()

,hash_id,price,price_czk,price_czk_unit,price_czk_name,area,lat,lon,locality,type,category,is_auction,exclusively_at_rk,category_main_cb,category_sub_cb,category_type_cb,company_id,company_name
0,3130370908,29789000,29789000,,Celková cena,1509,50.097913,14.363281,Praha 6 - Dejvice,1,3,False,0,3,19,1,3628,LUXENT - Exclusive Properties
1,2704906060,17888000,17888000,,Celková cena,1022,50.042608,14.54078,Praha 10 - Hostivař,1,3,False,1,3,19,1,NaN,NaN
2,280736844,22084000,22084000,,Celková cena,1027,50.051946,14.280384,Praha 5 - Sobín,1,3,False,0,3,19,1,17673,ARCHER reality Praha
3,1233892428,17784000,17784000,,Celková cena,827,50.051946,14.280384,Praha 5 - Sobín,1,3,False,0,3,19,1,17673,ARCHER reality Praha
4,767100492,34117000,34117000,,Celková cena,793,50.072035,14.58917,Praha 9 - Dolní Počernice,1,3,False,0,3,19,1,17673,ARCHER reality Praha


In [12]:
df.shape

(401, 18)

### Saving data

In [13]:
#df.to_parquet(exp_file_path+"/"+exp_file_name) 

SQLite:

In [14]:
con = sqlite3.connect('Estate_data.sqlite') # We must choose the name for our DB !

In [15]:
# Creates a table or appends if exists
df.to_sql(name = 'OFFERS_TABLE', con= con, index = False, if_exists = 'append')

401

In [16]:
# Closing the connection
con.close()

# Loading again: df = pd.read_sql('SELECT * FROM OFFERS_TABLE', con = con)